> **Current Scenario**

** There are 9172 confirmed cases according to WHO. Italy is a member state of European Union andvery popular for tourism. The first case was confirmed on 30th of january when two infected chinese tourists were found. Italy is the most infected country in Europe and second most affected place after China. Many italian visitors were confirmed that they are infected with Coronavirus after visiting different foreign countries. Many Countries from Asia, America, Europe found their local cases from Italy. In the very beginning only the northern area of italy was affected after a very short time it had spread to whole italy**

**Libraries**

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import datetime
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

**Reading Data**

In [ ]:
data=pd.read_csv("../input/covid19-in-italy/covid19_italy_region.csv")

In [ ]:
an_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")

In [ ]:
comp = pd.read_excel('/kaggle/input/covid19327/COVID-19-3.27-top30-500.xlsx')

In [ ]:
province = pd.read_csv("../input/covid19-in-italy/covid19_italy_province.csv")

 **Looking into data**

In [ ]:
an_data.head()

In [ ]:
an_data = an_data[an_data['country']=='Italy']
an_data.shape


**Age distribution of Confirmation**

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of Confirmation")
sns.kdeplot(data=an_data['age'], shade=True).set(xlim=(0))

> **Age**

**Here, the graph shows the age distribution of the infected people by gender. We can clearly see older people are more likely to be infected, especially older people with having lung disease and problems in their respiratory system. The age group of 40 to 50yr are more infected than the rest of the population in men. On the other hand age groups of 50yr to 70yr are more infected in womens. As Dr.Steven Gambert, professor of medicine and director of geriatrics at the University of Maryland School of Medicine says “ Older people have higher risk of underlying health conditions, older people are already under physical stress, and their immune systems, even if not significantly compromised, simply do not have the same “ability to fight viruses and bacteria”. As data says Italy has the oldest population across globe by count. According to EU statistics Italy has the lowest percentage of young people**.

**Gender Distribution of Confirmatioin**

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Gender')
an_data.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=an_data.groupby(['sex']).size().values,names=an_data.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

>  **Gender**

**The graph shows the age distribution of infected and confirmed people
by gender where men are more likely to die from coronavirus than
women. Research has found that middle-aged and older men are
having more chances to get infected by the virus. Researchers had
found that the infection rate among men and women is the same but the
death rate among men is 2.8% as compared with 1.7% for women.
As there are a few reasons men are more likely to die from the new
coronavirus. As because Women have a heightened immunity system
response. Scientists have explained the reason for more numbers of
infected people from Coronavirus in men over women. As in china
almost 50- 80% of men do smoking whereas only 2-3% of women do
smoking which affects the respiratory system of men over women.**

**Age distribution of the confirmation by gender**

In [ ]:
male_dead = an_data[an_data.sex=='male']
female_dead = an_data[an_data.sex=='female']

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the confirmation by gender")
sns.kdeplot(data=female_dead['age'], label="Women", shade=True).set(xlim=(0))
sns.kdeplot(data=male_dead['age'],label="Male" ,shade=True).set(xlim=(0))


In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(an_data,  size = 5)\
.map(plt.scatter, 'age', 'sex')\
.add_legend()
plt.title('Age vs Province',fontsize=40)
plt.xticks(fontsize=18)
plt.yticks(fontsize=28)


plt.show()

In [ ]:
data.head()

**Checking for Null Value**

In [ ]:
data.isna().sum()

**Description of Data**

In [ ]:
data.describe().T

**Tracking the Patient**

In [ ]:
data.shape

In [ ]:
clus=data.loc[:,['SNo','Latitude','Longitude']]
clus.head()

**Checking for number of cluster**

In [ ]:
K_clusters = range(1,15)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = data[['Latitude']]
X_axis = data[['Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Score vs Cluster')
plt.show()

**The score get cosntant after 4 clusters, so making more clusters will not help us. The value for k is 4 in this case**

In [ ]:
kmeans = KMeans(n_clusters = 4, init ='k-means++')
kmeans.fit(clus[clus.columns[1:3]])
clus['cluster_label'] = kmeans.fit_predict(clus[clus.columns[1:3]])
centers = kmeans.cluster_centers_
labels = kmeans.predict(clus[clus.columns[1:3]])

**Graphical representation of clusters**

In [ ]:
clus.plot.scatter(x = 'Latitude', y = 'Longitude', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.5)

**We will verify our clusters by putting values in world map by making use of folium library**

**Affected place in world map including Hospitalised , Confirm , Deaths and Recovery**

In [ ]:
 import folium
italy_map = folium.Map(location=[42.8719,12.5674 ], zoom_start=5,tiles='Stamen Toner')

for lat, lon,RegionName,TotalPositiveCases,Recovered,Deaths,TotalHospitalizedPatients in zip(data['Latitude'], data['Longitude'],data['RegionName'],data['TotalPositiveCases'],data['Recovered'],data['Deaths'],data['TotalHospitalizedPatients']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='red',
                      popup =('RegionName: ' + str(RegionName) + '<br>'
                    'TotalPositiveCases: ' + str(TotalPositiveCases) + '<br>'
                    'TotalHospitalizedPatients: ' + str(TotalHospitalizedPatients) + '<br>'
                      'Recovered: ' + str(Recovered) + '<br>'
                      'Deaths: ' + str(Deaths) + '<br>'),

                        fill_color='red',
                        fill_opacity=0.7 ).add_to(italy_map)
italy_map

**The most affected cities and regions early in Italy are Lombardy, and then Emilia-Romagna, Veneto, Marche, and Piemonte. Milan is the second most populous Italian city which is located in Lombardy. Other areas in Italy which are affected by coronavirus include Toscana, Campania, Lazio, Liguria, Friuli Venezia Giulia, Sicilia, Puglia, Umbria, Abruzzo, Trento, Molise, Calabria, Sardegna, Valle d’Aosta, Basilicata, and Bolzano. As italy was the fourth most affected coronavirus country
till last feb but now it has reached the maximum number of confirmed cases after China**

**Grouping Data According to  Region Name**

In [ ]:
data['Date'] = pd.to_datetime(data['Date']).dt.normalize()
daily = data.sort_values(['Date','Country','RegionName'])
latest = data[data.Date == daily.Date.max()]
latest.head()

In [ ]:
data_groupby_region = latest.groupby("RegionName")[['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients','TotalHospitalizedPatients']].sum().reset_index()
dgr = data_groupby_region 
dgr.head()

**Desciption of Grouped Data by Region**

In [ ]:
dgr.describe().T

**Test performed vs Region**

In [ ]:
fig = px.bar(dgr[['RegionName', 'TestsPerformed']].sort_values('TestsPerformed', ascending=False), 
             y="TestsPerformed", x="RegionName", color='RegionName', 
             log_y=True, template='ggplot2', title='Test Performed vs Region')
fig.show()


**As the graph shows the test performed in different regions of Italy. Lombardia has the maximum number(25k+) of tests performed as it is the most infected in cities. As a result the next graph shows that it has the maximum number(7280) of positive coronavirus patients. Veneto is the second most infected city here followed by some more countries like Emilia Romagna, Lazio, Marche, Toscana, Piemonte, Friuli V.G. ,Campania, Sicilia, Liguria, Puglia, P.A. Trento, Calabria, Umbria, Abruzzo, Sardegna, Molisa, Basilicata, Valle d'Aosta, P.A. Bolzano etc.
**

**Confirmed Cases vs Region**

In [ ]:
fig = px.bar(dgr[['RegionName', 'TotalPositiveCases']].sort_values('TotalPositiveCases', ascending=False), 
             y="TotalPositiveCases", x="RegionName", color='RegionName', 
             log_y=True, template='ggplot2', title='Confirmed Cases vs Region')
fig.show()


**There are more than 10,000 people who are infected through this virus. Italy is the most affected country in the world after China, with 827 deaths and 12,462 confirmed cases in almost three weeks. The government has restricted all flights from china. as because at the end of January after two Chinese tourists came down with coronavirus during a trip to Italy. At the time, it was hopefully the best measure which can
block the spread of the disease**

**Hospitalised Patient vs Region**

In [ ]:
fig = px.bar(dgr[['RegionName', 'TotalHospitalizedPatients']].sort_values('TotalHospitalizedPatients', ascending=False), 
             y="TotalHospitalizedPatients", x="RegionName", color='RegionName', 
             log_y=True, template='ggplot2', title='Hospitalised Patient vs Region')
fig.show()

**Recovery vs Region**

In [ ]:
fig = px.bar(dgr[['RegionName', 'Recovered']].sort_values('Recovered', ascending=False), 
             y="Recovered", x="RegionName", color='RegionName', 
             log_y=True, template='ggplot2', title='Revovery vs Region')
fig.show()


**Death vs Region Name**

In [ ]:
fig = px.bar(dgr[['RegionName', 'Deaths']].sort_values('Deaths', ascending=False), 
             y="Deaths", x="RegionName", color='RegionName', 
             log_y=True, template='ggplot2', title='Death vs Region')
fig.show()


In [ ]:
dgrs_el = dgr.sort_values(by=['TotalPositiveCases'],ascending = False)
dgrs_el.head()

**Test and Confirm vs Region**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgrs_el.RegionName, dgrs_el.TestsPerformed,label="Tests Performed")
plt.bar(dgrs_el.RegionName, dgrs_el.TotalPositiveCases,label="Confirm Cases")
plt.xlabel('Region')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Test and Confirm vs Region',fontsize = 35)

plt.show()

f, ax = plt.subplots(figsize=(80,30))
ax=sns.scatterplot(x="RegionName", y="TestsPerformed", data=dgrs_el,
             color="red",label = "Tests Performed")
ax=sns.scatterplot(x="RegionName", y="TotalPositiveCases", data=dgrs_el,
             color="blue",label = "Confirm Cases")
ax.xaxis.set_tick_params(labelsize=35)

plt.plot(dgrs_el.RegionName,dgrs_el.TestsPerformed,zorder=1,color="red")
plt.plot(dgrs_el.RegionName,dgrs_el.TotalPositiveCases,zorder=1,color="blue")

**Confirmed cases vs People Hospitalised**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgrs_el.RegionName, dgrs_el.TotalPositiveCases,label="Confirm Cases")
plt.bar(dgrs_el.RegionName, dgrs_el.TotalHospitalizedPatients,label="Hospitalized Patients")

plt.xlabel('Region')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confirm Cases vs People Hospitalised',fontsize= 35)
plt.show()

f, ax = plt.subplots(figsize=(40,20))

ax=sns.scatterplot(x="RegionName", y="TotalPositiveCases", data=dgrs_el,
             color="blue",label = "Confirm Cases")
ax=sns.scatterplot(x="RegionName", y="TotalHospitalizedPatients", data=dgrs_el,
             color="red",label = "Hospitalized Patients")
ax.xaxis.set_tick_params(labelsize=18)
plt.plot(dgrs_el.RegionName,dgrs_el.TotalPositiveCases,zorder=1,color="blue")
plt.plot(dgrs_el.RegionName,dgrs_el.TotalHospitalizedPatients,zorder=1,color="red")


**The graph shows statistical data direct from WHO. as the data says in Lombardia after 7,000 and more confirmed cases there are only approximately 4.5K people who are hospitalised. This has become a situation of crisis in italy. Hospital condition is becoming worse day by day. According to the doctors not every patient is getting proper and equal care and that is the main cause of multi fold spread of coronavirus. The whole country is locked down. Government has announced there will be no gathering, no sporting event and no travelling across the country just because of the high number of deaths in the country**

**Death and Recovery vs Region**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgrs_el.RegionName, dgrs_el.Recovered,label="Recovery")
plt.bar(dgrs_el.RegionName, dgrs_el.Deaths,label="Death")
plt.xlabel('Region')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Death and Recovery vs Region', fontsize= 35)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="RegionName", y="Recovered", data=dgrs_el,
             color="red",label = "Recovered")
ax=sns.scatterplot(x="RegionName", y="Deaths", data=dgrs_el,
             color="blue",label = "Deaths")
plt.plot(dgrs_el.RegionName,dgrs_el.Recovered,zorder=1,color="red")
plt.plot(dgrs_el.RegionName,dgrs_el.Deaths,zorder=1,color="blue")

**According to the graph recovery rate of the patient is very slow. There are some common reasons behind the rapid increase in numbers of people infected through coronavirus. According to the data, the number of hospitalized people is far less than the number of people infected through novel-Coronavirus. According to the geographical structure of Italy In Europe the cases have now been confirmed in every member nation of the European Union. Italy will remain totally locked down as its healthcare system struggles to cope, on the other hand the nearby countries like Germany and France report alarming spikes in daily cases**

In [ ]:
data['Date'] = pd.to_datetime(data['Date']).dt.normalize()
latest = data[data.Date == daily.Date.max()]

In [ ]:
temp = latest.loc[:,['Date','HospitalizedPatients','IntensiveCarePatients','TotalHospitalizedPatients','HomeConfinement','Recovered','Deaths','TotalPositiveCases','TestsPerformed']]
temp.head()

**Grouped by Provicne**

In [ ]:
province.head()

In [ ]:
provincegrp = province.groupby("ProvinceName")[["TotalPositiveCases"]].max().reset_index()

In [ ]:
fig = px.bar(provincegrp[['ProvinceName', 'TotalPositiveCases']].sort_values('TotalPositiveCases', ascending=False), 
             y="TotalPositiveCases", x="ProvinceName", color='ProvinceName', 
             log_y=True, template='ggplot2', title='Province vs Region')
fig.show()

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(province,  size = 30)\
.map(plt.scatter, 'RegionName', 'ProvinceName')\
.add_legend()
plt.title('Age vs Infection Reason',fontsize=30)
plt.xticks(fontsize=12)
plt.yticks(fontsize=15)
plt.show()

**Descipiton of Data grouped by Date**

In [ ]:
temp.describe().T

In [ ]:
data_groupby_date = latest.groupby("Date")[['Date','HospitalizedPatients','IntensiveCarePatients','TotalHospitalizedPatients','HomeConfinement','Recovered','Deaths','TotalPositiveCases','TestsPerformed']].sum().reset_index()
data_groupby_date

**Ratio and percentage of Confirmation, Deaths and Deaths, Recovery after Confirmation**

In [ ]:
ps_ts = float(data_groupby_date.TotalPositiveCases/data_groupby_date.TestsPerformed)
d_ts = float(data_groupby_date.Deaths/data_groupby_date.TestsPerformed)
r_ps = float(data_groupby_date.Recovered/data_groupby_date.TotalPositiveCases)
d_ps = float(data_groupby_date.Deaths/data_groupby_date.TotalPositiveCases)

In [ ]:
print("The percentage of Confirmation is "+ str(ps_ts*100) )
print("The percentage of Death is "+ str(d_ts*100) )
print("The percentage of Death after confirmation is "+ str(d_ps*100) )
print("The percentage of recovery after confirmation is "+ str(r_ps*100) )

In [ ]:
data_groupby_date1 = data.groupby("Date")[['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients','TotalHospitalizedPatients']].sum().reset_index()
dgd3 = data_groupby_date1
dgd3.head()

In [ ]:
dgd2 = dgd3

In [ ]:
dgd2["Date"]= dgd3["Date"].dt.strftime("%d-%m-%y") 
dgd2.head()

**Test vs Confirmed**


In [ ]:
dgd2 = dgd2.tail(14)

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgd2.Date, dgd2.TestsPerformed,label="Tests Performed")
plt.bar(dgd2.Date, dgd2.TotalPositiveCases,label="Confirm Cases")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Test Peroformed vs Confirmed Cases',fontsize = 35)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="Date", y="TestsPerformed", data=dgd2,
             color="red",label = "Tests Performed")
ax=sns.scatterplot(x="Date", y="TotalPositiveCases", data=dgd2,
             color="blue",label = "Confirm Cases")
plt.plot(dgd2.Date,dgd2.TestsPerformed,zorder=1,color="red")
plt.plot(dgd2.Date,dgd2.TotalPositiveCases,zorder=1,color="blue")

**Confirmed cases vs People Hospitalised**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgd2.Date, dgd2.TotalPositiveCases,label="Confirm Cases")
plt.bar(dgd2.Date, dgd2.TotalHospitalizedPatients,label="Hospitalized Patients")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confirmed Cases vs Hospitalised Cases',fontsize= 35)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="Date", y="TotalHospitalizedPatients", data=dgd2,
             color="red",label = "Hospitalized Patients")
ax=sns.scatterplot(x="Date", y="TotalPositiveCases", data=dgd2,
             color="blue",label = "Confirm Cases")
plt.plot(dgd2.Date,dgd2.TotalHospitalizedPatients,zorder=1,color="red")
plt.plot(dgd2.Date,dgd2.TotalPositiveCases,zorder=1,color="blue")

**Hospitalise vs Recevery and Death**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgd2.Date, dgd2.TotalHospitalizedPatients,label="Hospitaise Patients")
plt.bar(dgd2.Date, dgd2.Recovered,label="Recovery")
plt.bar(dgd2.Date, dgd2.Deaths,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Hospitalise vs Recovery vs Death',fontsize=30)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="Date", y="TotalHospitalizedPatients", data=dgd2,
             color="black",label = "Hospitalise Patients")
ax=sns.scatterplot(x="Date", y="Recovered", data=dgd2,
             color="red",label = "Recovery")
ax=sns.scatterplot(x="Date", y="Deaths", data=dgd2,
             color="blue",label = "Death")
plt.plot(dgd2.Date,dgd2.TotalHospitalizedPatients,zorder=1,color="black")
plt.plot(dgd2.Date,dgd2.Recovered,zorder=1,color="red")
plt.plot(dgd2.Date,dgd2.Deaths,zorder=1,color="blue")

**Confirm vs Recovery vs Death**

In [ ]:
plt.figure(figsize=(23,10))
plt.bar(dgd2.Date, dgd2.TotalPositiveCases,label="Confirm")
plt.bar(dgd2.Date, dgd2.Recovered,label="Recovery")
plt.bar(dgd2.Date, dgd2.Deaths,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confrim vs Recovery vs Death',fontsize=30)
plt.show()

f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="Date", y="TotalPositiveCases", data=dgd2,
             color="black",label = "Confirm")
ax=sns.scatterplot(x="Date", y="Recovered", data=dgd2,
             color="red",label = "Recovery")
ax=sns.scatterplot(x="Date", y="Deaths", data=dgd2,
             color="blue",label = "Death")
plt.plot(dgd2.Date,dgd2.TotalPositiveCases,zorder=1,color="black")
plt.plot(dgd2.Date,dgd2.Recovered,zorder=1,color="red")
plt.plot(dgd2.Date,dgd2.Deaths,zorder=1,color="blue")

**This graph gives an overview of the current situation of italy. There are more than 12,000 confirmed cases now. There is approximately equal number of deaths as of recovery. From the date the country has confirmed its first case of positive coronavirus it has been increasing exponentially. Till the date 11,Mar italy has become the second most infected country after China.**

In [ ]:
data_groupby_date1 = data.groupby("Date")[['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients','TotalHospitalizedPatients']].sum().reset_index()
dgd1 = data_groupby_date1
dgd1.head()

> **Prophet  Algorithm**

For Number of Test(Screening)

In [ ]:
pr_data_test = dgd1.loc[:,['Date','TestsPerformed']]
pr_data_test.columns = ['ds','y']
pr_data_test.head()

**Modeling**

In [ ]:
m = Prophet()
m.fit(pr_data_test)
future=m.make_future_dataframe(periods=15)
forecast_test=m.predict(future)
forecast_test

**Predicting**

In [ ]:
test = forecast_test.loc[:,['ds','trend']]
test = test[test['trend']>0]
test.head()
test=test.tail(15)
test.columns = ['Date','Screening']
test.head()


**Graphical Representation of Predicted Screening**

In [ ]:
fig_test = plot_plotly(m, forecast_test)
py.iplot(fig_test) 

fig_test = m.plot(forecast_test,xlabel='Date',ylabel='Scrrening Count')

In [ ]:
figure_test=m.plot_components(forecast_test)


**The graph predicts the total number of screening required to take the condition of Italy under control. When there will be more than 14Lakhs of screening till Mar,2021 then the situation will be under control. As the graph is linear so European governments are supposed to work accordingly.**

**Confirmed Cases**

**Making Data ready for algorithm**

In [ ]:
pr_data_cm = dgd1.loc[:,['Date','TotalPositiveCases']]
pr_data_cm.columns = ['ds','y']
pr_data_cm.head()

**Modelling**

In [ ]:
m=Prophet()
m.fit(pr_data_cm)
future=m.make_future_dataframe(periods=15)
forecast_cm=m.predict(future)
forecast_cm

**Predicting**

In [ ]:
cnfrm = forecast_cm.loc[:,['ds','trend']]
cnfrm = cnfrm[cnfrm['trend']>0]
cnfrm=cnfrm.tail(15)
cnfrm.columns = ['Date','Confirm']
cnfrm.head()

**Graphical Representation of Predicted Confiramtion**

In [ ]:
fig_cm = plot_plotly(m, forecast_cm)
py.iplot(fig_cm) 

fig_cm = m.plot(forecast_cm,xlabel='Date',ylabel='Confirmed Count')

In [ ]:
figure_cm=m.plot_components(forecast_cm)


Predicting the number of cases confirmed in Italy till Feb, 2021. Therewill be more than 2.5 Lakhs people who will be affected by coronavirus.

**Recovery**

**Making Data Ready for algorithm**

In [ ]:
pr_data_r = dgd1.loc[:,['Date','Recovered']]
pr_data_r.columns = ['ds','y']
pr_data_r.head()

**Modelling**

In [ ]:
m=Prophet()
m.fit(pr_data_r)
future=m.make_future_dataframe(periods=15)
forecast_r=m.predict(future)
forecast_r

**Predicting**

In [ ]:
rec = forecast_r.loc[:,['ds','trend']]
rec = rec[rec['trend']>0]
rec=rec.tail(15)
rec.columns = ['Date','Recovery']
rec.head()

**Graphical Representation of Predicted Recovery**

In [ ]:
fig_r = plot_plotly(m, forecast_r)
py.iplot(fig_r) 

fig_r = m.plot(forecast_r,xlabel='Date',ylabel='Recovery Count')

In [ ]:
figure_r=m.plot_components(forecast_r)


**For   Deaths**

**Making data ready for algorithm**

In [ ]:
pr_data_d = dgd1.loc[:,['Date','Deaths']]
pr_data_d.columns = ['ds','y']
pr_data_d.head()

**Modelling**

In [ ]:
m=Prophet()
m.fit(pr_data_d)
future=m.make_future_dataframe(periods=15)
forecast_d=m.predict(future)
forecast_d

**Predicting**

In [ ]:
dth = forecast_d.loc[:,['ds','trend']]
dth = dth[dth['trend']>0]
dth=dth.tail(15)
dth.columns = ['Date','Death']
dth.head()


**Graphical Representation of Predicted Death**

In [ ]:
fig_d = plot_plotly(m, forecast_d)
py.iplot(fig_d) 

fig_d = m.plot(forecast_d,xlabel='Date',ylabel='Deaths Count')

In [ ]:
figure_d=m.plot_components(forecast_d)


**The current situation is not under control. According to current data recovery percent of confirmed patients is 53% whereas death is 47% respectively. But the prediction says there will approximately 25000 people who will be recovered and around 14000 people will be dead till Feb 2021. And rest will be in isolation.**

**How future looks like**

In [ ]:
prediction = test
prediction['Confirm'] = cnfrm.Confirm
prediction['Recover'] = rec.Recovery
prediction['Death'] = dth.Death

In [ ]:
prediction.head()

**Future Ratios**

In [ ]:
pps_pts = float(prediction.Confirm.sum()/prediction.Screening.sum())
pd_pts = float(prediction.Death.sum()/prediction.Screening.sum())
pr_pps = float(prediction.Recover.sum()/prediction.Confirm.sum())
pd_pps = float(prediction.Death.sum()/prediction.Confirm.sum())

In [ ]:
print("The percentage of  Predicted Confirmation is "+ str(pps_pts*100) )
print("The percentage of Predicted Death is "+ str(pd_pts*100) )
print("The percentage of Predicted Death after confirmation is "+ str(pd_pps*100) )
print("The percentage of Predicted recovery after confirmation is "+ str(pr_pps*100) )

**Comparision with other countries**

In [ ]:
comp.head()

In [ ]:
comp_table = pd.DataFrame(comp.describe().T)
comp_table

In [ ]:
comp = comp.loc[:,["Canada","US","Italy","France","Spain","Austria"]]

In [ ]:
comp.plot()

> **Prevention**
To avoid the critical situation people are suggested to do following
things
*  Avoid contact with people who are sick.
*  Avoid touching your eyes, nose, and mouth.
*  Stay home when you are sick.
*  Cover your cough or sneeze with a tissue, then throw the tissue in
the trash.
*  Clean and disinfect frequently touched objects and surfaces using
a regular household
*  Wash your hands often with soap and water, especially after going
to the bathroom; before eating; and after blowing your nose,
coughing, or sneezing.If soap and water are not readily available, use an alcohol-based
hand sanitizer.